In [2]:
from lmptools import Dump, DumpCallback, SimulationBox, Atom, DumpSnapshot
from pydantic import parse_obj_as
from lmptools.exceptions import SkipSnapshot
import random

In [3]:
def dump_file():
    # Create a dump file
    filename = "dump.test.lammpstrj"
    num_timesteps = random.randint(10, 100)

    snapshots = []
    with open(filename, "w") as f:
        for _ in range(num_timesteps):
            timestep = random.randint(10000, 100000)
            num_atoms = random.randint(10, 100)
            box_length = random.random()
            xlo = -box_length
            xhi = box_length
            ylo = -box_length
            yhi = box_length
            zlo = -box_length
            zhi = box_length

            box = SimulationBox(xlo=xlo, xhi=xhi, ylo=ylo, yhi=yhi,
                                zlo=zlo, zhi=zhi, xprd='pp', yprd='pp', zprd='pp')

            dump_colnames: str = ""
            if random.random() <= 0.25:
                dump_colnames = "id type mol x y z ix iy iz"
            elif random.random() > 0.25 or random.random() <= 0.5:
                dump_colnames = "id type mol x xu xsu yu z ix iy"
            else:
                dump_colnames = "id type mol x y zu zs z"

            f.write("ITEM: TIMESTEP\n")
            f.write(f"{timestep}\n")
            f.write("ITEM: NUMBER OF ATOMS\n")
            f.write(f"{num_atoms}\n")
            f.write("ITEM: BOX BOUNDS pp pp pp\n")
            f.write(f"{xlo} {xhi}\n")
            f.write(f"{ylo} {yhi}\n")
            f.write(f"{zlo} {zhi}\n")

            f.write(f"ITEM: ATOMS {dump_colnames}\n")
            atoms = []
            for _ in range(num_atoms):
                entry = {}
                random_numbers = [random.random()+random.randint(100, 1000) for _ in range(len(dump_colnames))]
                for key, value in zip(dump_colnames.split(), random_numbers):
                    entry[key] = value
                atom = parse_obj_as(Atom, entry)
                atoms.append(atom)
                f.write(" ".join([str(atom.__dict__[key]) for key in dump_colnames.split()])+"\n")

            snapshots.append(DumpSnapshot(timestamp=timestep, natoms=num_atoms, box=box, atoms=atoms, unwrapped=False))
    f.close()


# Create the dump file
dump_file()

### Callack to skip all snapshots with timestamp < 6000

In [16]:
class MyCallback(DumpCallback):
    def on_snapshot_parse_time(self, timestamp: int):
        if timestamp < 100:
            raise SkipSnapshot(f"snap {timestamp} skipped")

In [17]:
d = Dump("dump.test.lammpstrj", callbacks=MyCallback())

### Parse the dump file

In [18]:
d.parse()